In [1]:
import pandas as pd 
import numpy as np
import re
import transformers




In [2]:
df = pd.read_csv('df_with_combat_v1.csv')

C:\Users\sonso\AppData\Local\Temp\ipykernel_26392\1842741491.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_with_combat_v1.csv')


In [3]:
pd.set_option('display.max_colwidth', None)
dm_df = df[df['speaker'] == '<DM>']

In [4]:
def loc_window(df,t, idx, size):
    if t == 'iloc':
        return df.iloc[idx-size:idx+size+1]
    elif t == 'loc':
        return df.loc[idx-size:idx+size+1]
    return 'Invalid type'

### regex label some of the data


In [84]:

def simple_label(text):
    # check for narratives like phrases
    if re.search(r'\bAs (?:he|she|they|it|the|you)\b(?: [a-zA-Z]+){0,5}', text, re.IGNORECASE):
        return 'IC'  # IC
    if re.search(r'\d+d\d+|\[\d+\]|\bHit\b|\bCrit\b|\bDamage\b|\bAttack\b', text, re.IGNORECASE) and len(text) < 250:
        return 'OOC'  # OOC
    if re.search(r'"[^"]*"', text) and len(text) > 250:
        return 'IC'
    if re.search(r'\btwitch\b', text, re.IGNORECASE):
        return 'OOC'
    if len(text) <= 250:
        return 'OOC'
    return "IC"



In [ ]:
from tqdm import tqdm

tqdm.pandas(desc="Processing Simple Label")
dm_df['IC_OOC'] = dm_df['utterance'].progress_apply(simple_label)

### Manual label

In [70]:
def pprint(text):
    text = text.replace(r'.', '.\n')
    text = text.replace(r',', ',\n')
    print(text)

In [71]:
from IPython.display import clear_output
manual_data = []
manual_df = dm_df.iloc[500:3000].copy()

import sys


for i, row in tqdm(manual_df.iterrows(), total=manual_df.shape[0], desc="Manual Labeling"):
    idx = row.name
    speaker = row['speaker']
    utterance = row['utterance']
    ic_ooc = row['IC_OOC']

    pprint(f"Index: {idx}")
    pprint(f"Speaker: {speaker}")
    pprint(f"Utterance: {utterance}")
    pprint(f"IC/OOC: {ic_ooc}")

    select = input('Type 1 for IC, 2 for OOC, 3 for skip: ')

    clear_output()

    if select == '1':
        manual_data.append((idx, utterance, 'IC'))
    elif select == '2':
        manual_data.append((idx, utterance, 'OOC'))
    elif select == '3':
        continue
    elif select == 'stop':
        break
    else:
        print("Invalid selection. Please try again.")
    


Manual Labeling:  20%|██        | 507/2500 [41:56<2:44:50,  4.96s/it]


In [72]:
manual_data

[(1642,
  'That means the damage dice you double. Roll for damage, double that, and add your modifier after that.',
  'OOC'),
 (1648,
  "You guys hear this cacophonous, large blasting sound, a flash of blue energy and sparks shoot out the back of the weapon. It actually lifts Percy off the ground for a second and he shifts himself from the sheer blast. This streaking, heated bullet, reddish, white hot, goes cascading through the air, plunges into the side of the ogre's shoulder with a smattering of blood splattering against the cave behind it. It's actually moved back a step and has to catch itself. Slams its club into the ground. That ends your turn?",
  'IC'),
 (1654, 'This is the entanglement, by the way.', 'OOC'),
 (1657,
  'Everything there is fighting from the entanglement. Most of these goblins are currently inside of it.',
  'IC'),
 (1659, 'Both of them?', 'OOC'),
 (1661, 'You moved up to here? All righty.', 'OOC'),
 (1665,
  'These guys here are entangled. These front goblins 

In [5]:
# labeled_df = pd.DataFrame(manual_data, columns=['index', 'utterance', 'IC_OOC'])

# labeled_df.to_csv('labeled_icooc_data.csv', index=False)

labeled_df = pd.read_csv('labeled_icooc_data.csv')

In [7]:
loc_window(labeled_df, 'iloc', 300, 5)

,index,utterance,IC_OOC
295,2882,And an entryway that is currently open and a warmly-lit interior welcomes you.,IC
296,2915,That's not what it's called. (group laughs),OOC
297,2917,"It's the Value of Valor guys, come on! (whines)",OOC
298,2921,A giant boulder falls out of nowhere and crushes Grog. (group laughs),IC
299,2922,"So you make your way inside. Suddenly, hit with the smell of heavy incense and jasmine. This is the equivalent of a Psychic Eye Bookshop.",IC
300,2936,"""Dwarves have a hard time buying from those who are not dwarves. But I've made my established connections with business individuals. So, what can I help you with?""",IC
301,2939,"""Of which intensity are you looking for?""",IC
302,2941,"""So looking for a superior healing potion.""",IC
303,2943,"""I've only got two of those currently.""",IC
304,2946,"""I can sell you the two superior healing potions.""",IC


In [8]:
labeled_df.tail()

,index,utterance,IC_OOC
340,3194,"You make your way down to the bottom ring of Kraghammer. To the mines proper. The miners have gone back to work, and from what you can see, there's just an all around crazy business with people cleaning out, still trying to clean out the ichor that's on the ground left over from the battle you guys had the day before. You make your way to the front of the mines, and immediately three of the carvers that are standing at the front approach as you begin to walk, and then upon noticing you, kind of like, step back and put their weapons to the side.",IC
341,3201,I assume Trinket comes with you unless you say otherwise.,OOC
342,3207,"And fair play to the gnome, but they're dwarves.",OOC
343,3210,"You make your way into the mine proper. It goes back for a good 10 minutes of walking and of winding. You can see the actual track of the minetrack leading around. You come upon a couple of dwarves that look over their shoulders begrudgingly at you guys as you pass going, ""Hey? Eh, shh,"" and go right back into mining whatever remains in the inside of this. It looks like this portion of the mine has been nearly picked clean. The dwarves that are in there are probably just wasting time to get their pay for the day. You go further back and eventually it opens up. I think I have the map available.",IC
344,3218,"We can put it on there perhaps. For the most part I'll be describing it as you go, so it's not too important.",OOC


## simple classifier BERT


In [9]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

In [10]:
model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:

dataset = Dataset.from_pandas(labeled_df[['utterance', 'IC_OOC']])

# map IC_OOC to labels column to label feature


dataset = dataset.map(lambda x: {'labels': 1 if x['IC_OOC'] == 'IC' else 0}, remove_columns=['IC_OOC'])

dataset

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

Dataset({
    features: ['utterance', 'labels'],
    num_rows: 345
})

In [13]:
# Split the dataset into train and test sets
data_to_model = dataset.train_test_split(test_size=0.125, shuffle=True, seed=42)

In [14]:
data_to_model


DatasetDict({
    train: Dataset({
        features: ['utterance', 'labels'],
        num_rows: 301
    })
    test: Dataset({
        features: ['utterance', 'labels'],
        num_rows: 44
    })
})

In [15]:
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True, padding=True, max_length=512)


tokenized_dataset = data_to_model.map(preprocess_function, batched=True)

Map:   0%|          | 0/301 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['utterance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 301
    })
    test: Dataset({
        features: ['utterance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 44
    })
})

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
import evaluate

accuracy = evaluate.load("accuracy")



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
id2label = {0: "OOC", 1: "IC"}
label2id = {"OOC": 0, "IC": 1}

In [20]:

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    output_dir="ICOOC_model_manual",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
# load the model from the checkpoint
# model = AutoModelForSequenceClassification.from_pretrained("ICOOC_model/checkpoint-10938")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gannubvv (gannubvv-chulalongkorn-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.475023,0.863636
2,No log,0.331868,0.931818
3,No log,0.186416,0.954545
4,No log,0.164617,0.954545
5,No log,0.149849,0.954545


TrainOutput(global_step=95, training_loss=0.2979020771227385, metrics={'train_runtime': 20.1386, 'train_samples_per_second': 74.732, 'train_steps_per_second': 4.717, 'total_flos': 86053357675740.0, 'train_loss': 0.2979020771227385, 'epoch': 5.0})

In [22]:
model = AutoModelForSequenceClassification.from_pretrained("ICOOC_model_manual/checkpoint-57")

In [23]:
text = 'Okay. As you concentrate and these kind of shimmering bands of arcane energy wave off your hands toward the enchantment, they impact and dissipate without effect.'

In [24]:
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

In [25]:
import torch

In [26]:
def infer(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predicted_class = id2label[predicted_class_id]
    return predicted_class

In [ ]:
test_df = dm_df[['utterance', 'IC_OOC']].iloc[100000:]

In [ ]:

test_df['predicted'] = test_df['utterance'].progress_apply(infer)

In [ ]:
test_df

In [ ]:
loc_window(test_df, 379276, 50)

In [32]:
from tqdm import tqdm
tqdm.pandas(desc="Processing Simple Label")

In [ ]:
# to_save_df = pd.read_csv('df_with_combat_v1.csv')
# to_save_df['IC_OOC'] = to_save_df['utterance'].progress_apply(in    `fer)
# to_save_df.to_csv('df_combat_icooc_v2_manual.csv', index=False)

C:\Users\sonso\AppData\Local\Temp\ipykernel_26392\30534623.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  to_save_df = pd.read_csv('df_with_combat_v1.csv')
Processing Simple Label: 100%|██████████| 398682/398682 [1:47:05<00:00, 62.05it/s]  


In [35]:

df_with_icooc = pd.read_csv('df_combat_icooc_v2_manual.csv')


C:\Users\sonso\AppData\Local\Temp\ipykernel_26392\69795369.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_with_icooc = pd.read_csv('df_combat_icooc_v2_manual.csv')


In [39]:
loc_window(df_with_icooc, 'iloc', 179276, 50)

,fileID,number,speaker,utterance,item,owner,in_combat,IC_OOC
179226,C1E077,1596,<DM>,"Staying in melee range of the creature or do you want to leave its melee range? It will get a strike at you if you do. Actually, it will not because it already stuck at Percy so you can go there, you're fine. All right, that ends your turn. Grog, you're up.",NaN,NaN,True,IC
179227,C1E077,1597,<PLAYER1>,I see the giant eyeing Scanlan and I'm like: I thought you said I was your charge. You are not cheating on me this early in our relationship. And I take both of my attacks at general hot fire giant's face. Reckless. Great Weapon Master.,NaN,NaN,True,IC
179228,C1E077,1598,<DM>,Go for it.,NaN,NaN,True,OOC
179229,C1E077,1599,<PLAYER1>,30 minus five is 25!,NaN,NaN,True,OOC
179230,C1E077,1600,<DM>,Hits.,NaN,NaN,True,OOC
...,...,...,...,...,...,...,...,...
179322,C1E077,1692,<PLAYER4>,Natural 20. Two natural 20's!,NaN,NaN,True,OOC
179323,C1E077,1693,<DM>,"All righty, and the wyvern rider is going to take a swing at you.",NaN,NaN,True,IC
179324,C1E077,1694,<PLAYER4>,"Sure, sure.",NaN,NaN,True,OOC
179325,C1E077,1695,<DM>,That is a 22.,NaN,NaN,True,OOC
